In [19]:
import backtesting
from backtesting import Strategy, Backtest
import pandas as pd
import keras
import numpy as np
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Constants
# List of columns to scale
columns_to_scale = ['Open', 'High', 'Low','Close', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)  
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3

# Target index ('Close')
close_index = 22

class CNN_Class_S1(Strategy):
    def init(self):
        self.trade_position = None
        self.days_since_position = 0

    def next(self):
        if self.data.Predictions > 0 and self.trade_position != 1:
            self.buy()
            self.trade_position = 1
            self.days_since_position = 0
        elif self.data.Predictions == 0 and self.trade_position != 0:
            self.sell()
            self.trade_position = 0
            self.days_since_position = 0

        # Close position if it's been S2 for a day
        if self.days_since_position == 1:
            if self.trade_position == 1:
                self.sell()
            else:
                self.buy()
            self.trade_position = None

        self.days_since_position += 1

class CNN_Class_S2(Strategy):
    def init(self):
        self.trade_position = None

    def next(self):
        # Long position logic
        if self.data.Predictions > 0 and self.trade_position != 1:
            if self.trade_position == 0:  
                self.buy()
            self.buy()  # S2 long position
            self.trade_position = 1
        # Short position logic
        elif self.data.Predictions == 0 and self.trade_position != 0:
            if self.trade_position == 1:  
                self.sell()
            self.sell() 
            self.trade_position = 0

# Full data #

In [20]:
seed_value = 123
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/BTC_FINAL_df_class.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/CNN/CNN_CLASS/best_CNN3DCLASS.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))


# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()

# merge the stats
# Merge the stats
full_stats = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])

full_stats = full_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)


full_stats.to_latex('cnn_full_stats.tex', float_format="%.2f")
full_stats

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


,S1,S2
Duration,340 days 00:00:00,340 days 00:00:00
Return [%],-1.275055,-9.065519
Buy & Hold Return [%],150.974205,150.974205
Return (Ann.) [%],-1.36418,-9.671695
Volatility (Ann.) [%],36.870213,34.109252
Sharpe Ratio,0.0,0.0
# Trades,343,145
Win Rate [%],46.06414,47.586207


# Bear Markets #

## Bear 1 ##

In [21]:
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
bear_1 = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bear_market_1_classification.csv')

# Load model
bear_1_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/CNN/CNN_CLASS/best_CNN-3D-CLASS-bear_market_1.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
br_1_backtest_data = bear_1[col]
training_size = int(len(bear_1) * 0.9)
br_1_backtest_data = br_1_backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
bear_1.set_index('Date', inplace=True)


# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
bear_1[columns_to_scale] = scaler.fit_transform(bear_1[columns_to_scale])

# Train and test data
training_size = int(len(bear_1) * 0.9)
test_data = bear_1[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))

# Make predictions
br_1_predictions = bear_1_model.predict(X_test)
br_1_test_pred = (br_1_predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
br_1_test_pred = np.concatenate([np.zeros((window_size, 1)), br_1_test_pred])

# Add the predictions to the test data
br_1_backtest_data['Predictions'] = br_1_test_pred.astype(int)

# Erase the first 3 values
br_1_backtest_data = br_1_backtest_data[window_size:]

# set date as index
br_1_backtest_data.set_index('Date', inplace=True)
br_1_backtest_data.index = pd.to_datetime(br_1_backtest_data.index)

# Buy and sell for 1 day
btS1 = Backtest(br_1_backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
statsS1 = btS1.run()


# Buy and keep S2 until other side prediction
bt = Backtest(br_1_backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats = bt.run()


# Merge the stats
full_stats_br1 = pd.concat([statsS1, stats], axis=1, keys=['S1', 'S2'])

full_stats_br1

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


,S1,S2
Start,2018-11-12 00:00:00,2018-11-12 00:00:00
End,2018-12-15 00:00:00,2018-12-15 00:00:00
Duration,33 days 00:00:00,33 days 00:00:00
Exposure Time [%],94.117647,94.117647
Equity Final [$],114809.906698,146611.992035
Equity Peak [$],131066.971614,146611.992035
Return [%],14.809907,46.611992
Buy & Hold Return [%],-49.197543,-49.197543
Return (Ann.) [%],340.449492,5979.647478
Volatility (Ann.) [%],563.887333,3715.635056


## Bear Market 2 ##

In [22]:
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bear_market_2_classification.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/CNN/CNN_CLASS/best_CNN-3D-CLASS-bear_market_2.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))


# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()

# merge the stats
# Merge the stats
full_stats_br2 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])

full_stats_br2

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


,S1,S2
Start,2022-10-17 00:00:00,2022-10-17 00:00:00
End,2022-11-21 00:00:00,2022-11-21 00:00:00
Duration,35 days 00:00:00,35 days 00:00:00
Exposure Time [%],94.444444,94.444444
Equity Final [$],80731.932383,115067.911805
Equity Peak [$],102428.615784,117123.170595
Return [%],-19.268068,15.067912
Buy & Hold Return [%],-19.249758,-19.249758
Return (Ann.) [%],-88.583234,314.961862
Volatility (Ann.) [%],8.662039,315.798022


In [23]:
# Merge the stats
cnn_bear_stats = pd.concat([full_stats_br1, full_stats_br2], axis=1, keys=['Bear_Market_1', 'Bear_Market_2'])


# Erase unecessary rows
cnn_bear_stats = cnn_bear_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)

# Aggregate the stats
# Calculate the averages for each strategy
cnn_bear_stats['S1_Avg'] = (cnn_bear_stats[('Bear_Market_1', 'S1')] + cnn_bear_stats[('Bear_Market_2', 'S1')]) / 2
cnn_bear_stats['S2_Avg'] = (cnn_bear_stats[('Bear_Market_1', 'S2')] + cnn_bear_stats[('Bear_Market_2', 'S2')]) / 2

cnn_bear_stats.to_latex('cnn_bear_stats.tex', float_format="%.2f", multicolumn=True)


# Bull Markets #

## Bull market 1 ##

In [24]:
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bull_market_1_classification.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/CNN/CNN_CLASS/best_CNN-3D-CLASS-bull_market_1.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))


# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()


# merge the stats
# Merge the stats
full_stats_bl1 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])
full_stats_bl1

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


,S1,S2
Start,2017-09-04 00:00:00,2017-09-04 00:00:00
End,2017-12-17 00:00:00,2017-12-17 00:00:00
Duration,104 days 00:00:00,104 days 00:00:00
Exposure Time [%],98.113208,79.245283
Equity Final [$],108395.480195,0.0
Equity Peak [$],137549.25149,126730.330865
Return [%],8.39548,-100.0
Buy & Hold Return [%],351.827192,351.827192
Return (Ann.) [%],32.344381,0.0
Volatility (Ann.) [%],201.618489,32699.866962


## Bull Market 2 ##

In [25]:
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bull_market_2_classification.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/CNN/CNN_CLASS/best_CNN-3D-CLASS-bull_market_2.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))


# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()

# merge the stats
# Merge the stats
full_stats_bl2 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])

full_stats_bl2

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


,S1,S2
Start,2021-07-30 00:00:00,2021-07-30 00:00:00
End,2021-11-10 00:00:00,2021-11-10 00:00:00
Duration,103 days 00:00:00,103 days 00:00:00
Exposure Time [%],98.095238,98.095238
Equity Final [$],150364.832716,106970.714334
Equity Peak [$],150364.832716,121889.8011
Return [%],50.364833,6.970714
Buy & Hold Return [%],53.887508,53.887508
Return (Ann.) [%],318.519111,26.680138
Volatility (Ann.) [%],251.439429,69.742421


In [26]:
# Merge the stats
cnn_bull_stats = pd.concat([full_stats_bl1, full_stats_bl2], axis=1, keys=['Bull_Market_1', 'Bull_Market_2'])


# Erase unecessary rows
cnn_bull_stats = cnn_bull_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)

# Aggregate the stats
# Calculate the averages for each strategy
cnn_bull_stats['S1_Avg'] = (cnn_bull_stats[('Bull_Market_1', 'S1')] + cnn_bull_stats[('Bull_Market_2', 'S1')]) / 2
cnn_bull_stats['S2_Avg'] = (cnn_bull_stats[('Bull_Market_1', 'S2')] + cnn_bull_stats[('Bull_Market_2', 'S2')]) / 2

cnn_bull_stats.to_latex('cnn_bull_stats.tex', float_format="%.2f", multicolumn=True)

cnn_bull_stats

Bull_Market_1                     \
                                      S1                 S2   
Duration               104 days 00:00:00  104 days 00:00:00   
Return [%]                       8.39548             -100.0   
Buy & Hold Return [%]         351.827192         351.827192   
Return (Ann.) [%]              32.344381                0.0   
Volatility (Ann.) [%]         201.618489       32699.866962   
Sharpe Ratio                    0.160424                0.0   
# Trades                             104                  1   
Win Rate [%]                   44.230769                0.0   

                           Bull_Market_2                     \
                                      S1                 S2   
Duration               103 days 00:00:00  103 days 00:00:00   
Return [%]                     50.364833           6.970714   
Buy & Hold Return [%]          53.887508          53.887508   
Return (Ann.) [%]             318.519111          26.680138   
Volatility (Ann.) [%]         251.439429          69.742421   
Sharpe Ratio                    1.266783           0.382553   
# Trades                             103                  7   
Win Rate [%]                   55.339806          14.285714   

                                  S1_Avg             S2_Avg  
                                                             
Duration               103 days 12:00:00  103 days 12:00:00  
Return [%]                     29.380156         -46.514643  
Buy & Hold Return [%]          202.85735          202.85735  
Return (Ann.) [%]             175.431746          13.340069  
Volatility (Ann.) [%]         226.528959       16384.804692  
Sharpe Ratio                    0.713603           0.191276  
# Trades                           103.5                4.0  
Win Rate [%]                   49.785288           7.142857